In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import math
import linmix

'''This code takes our sample and calculates a regression line that statistically factors in null data points.'''

#1. This section imports the necessary columns from the csv file into arrays
csv = np.genfromtxt ('RadioDetectionTable_Limits.csv', delimiter=",")
spectype = csv[1:,2] 
LV = csv[1:,7]
ysig = csv[1:,8]
xsig = csv[1:,9]
delta = csv[1:,10]
delta = delta.astype(int)
delta = delta == 0

In [ ]:
#THIS RUNS WITHOUT LV IN LOG FORMAT
#--. Runs the linmix algorithm (uses markov chain monte carlo)
#--. Plots two subplots. The right is the sample, and the left is the sample with error measurements and linear regression running through it.
# lm = linmix.LinMix(spectype, LV, xsig, ysig, K=2)
# lm.run_mcmc(silent=True)

# fig = plt.figure(figsize=(10,4))
# ax = fig.add_subplot(121)
# ax.scatter(spectype, LV)
# ax.set_xlabel('Spectral Type')
# ax.set_ylabel('L$_V$ [erg s$^{-1}$ Hz$^{-1}$]')
# # ax.set_yscale('log')
# ax.set_xlim(6,29)
# ax.set_ylim(400000000000,650000000000000)
# ax = fig.add_subplot(122)
# ax.scatter(spectype, LV, alpha=0.5)
# #ax.errorbar(spectype, LV, xerr=xsig, yerr=ysig, ls=' ', alpha=0.5)
# for i in xrange(0, len(lm.chain), 100): #outputs the value for each chain iteration
#     xs = np.arange(6,31)
#     ys = lm.chain[i]['alpha'] + xs * lm.chain[i]['beta']
#     ax.plot(xs, ys, color='r', alpha=0.07)
# #     return alpha_av = np.mean(lm.chain[i]['alpha'])
# #     return beta_av = np.mean(lm.chain[i]['beta'])
# # ys = alpha_av + xs * beta_av
# # ax.plot(xs, ys, color='k')
# ax.set_xlabel('Spectral Type')
# ax.set_ylabel('L$_V$ [erg s$^{-1}$ Hz$^{-1}$]')
# # ax.set_yscale('log')
# ax.set_xlim(6,29)
# # ax.set_ylim(400000000000,1000000000000000)
# ax.set_ylim(400000000000,650000000000000)
# ax.plot([6,29,29,6,6], [400000000000,400000000000,1000000000000000,1000000000000000,400000000000], color='k')
# fig.tight_layout()
# fig.savefig('radio_linmixanalysis.png')

In [ ]:
linmix.LinMix?

In [ ]:
#2. Converts Radio flux to log_10 values of radio flux
LV=np.log10(LV)

In [ ]:
# print LV

In [ ]:
#3. Runs the linmix algorithm (uses markov chain monte carlo)

lm = linmix.LinMix(spectype, LV, xsig=None, ysig=None, K=2)
lm.run_mcmc(silent=True)

In [ ]:
#4. This is the same action as 4 above, just the flux is in log_10
fig = plt.figure(figsize=(10,4))
ax = fig.add_subplot(121)
ax.scatter(spectype, LV)
ax.set_xlabel('Spectral Type')
ax.set_ylabel('log$_{10}$L$_V$ [erg s$^{-1}$ Hz$^{-1}$]')
ax.set_xlim(6,29)
ax.set_ylim(10,18)
labels = ['M6','','M8','','L0','','L2','','L4','','L6','','L8','',
          'T0','','T2','','T4','','T6','','T8','']
ylabels = ['','11','','13','','15','','17','']
plt.xticks(np.arange(6,30,1), labels)
plt.yticks(np.arange(10,19,1), ylabels)
ax = fig.add_subplot(122)
ax.scatter(spectype, LV, alpha=0.5)
#ax.errorbar(spectype, LV, xerr=xsig, yerr=ysig, ls=' ', alpha=0.5)
for i in xrange(0, len(lm.chain), 100): #outputs the value for each chain iteration
    xs = np.arange(6,31)
    ys = lm.chain[i]['alpha'] + xs * lm.chain[i]['beta']
    ax.plot(xs, ys, color='r', alpha=0.07)
alpha_av = lm.chain['alpha'].mean()
beta_av = lm.chain['beta'].mean()
ys = alpha_av + xs * beta_av
ax.plot(xs, ys, color='k')
ax.set_xlabel('Spectral Type')
ax.set_ylabel('log$_{10}$L$_V$ [erg s$^{-1}$ Hz$^{-1}$]')
ax.set_xlim(6,29)
ax.set_ylim(10,18)
plt.xticks(np.arange(6,30,1), labels)
plt.yticks(np.arange(10,19,1), ylabels)
fig.tight_layout()
fig.savefig('radio_linmixanalysislogged.png')
plt.close(fig)

In [ ]:
#5. This runs the mcmc code for only confirmed detections
# Can be commented out, creates arrays that will only show upper limits
spectype_conf=spectype[delta]
LV_conf=LV[delta]

lmconf = linmix.LinMix(spectype_conf, LV_conf, xsig=None, ysig=None, K=2)
lmconf.run_mcmc(silent=True)

In [ ]:
#6. This runs the mcmc code with delta indicating upper limits. 0=confirmed detections 1=null detections
lmcens  = linmix.LinMix(spectype, LV, xsig, ysig, delta=delta, K=2)
lmcens.run_mcmc(silent=True)

In [ ]:
#7. This runs the new plots with upper limits factored in

#Defines error arrays
ysig_b=np.multiply(LV, 0.1)
ysig_upper= LV + ysig_b
ysig_lower= LV - ysig_b 
ysig_b=ysig_upper - ysig_lower
ysig_b=np.log10(ysig_b)

#Defines not delta to graph correct arrays
notdelta = np.logical_not(delta)
labels = ['M6','','M8','','L0','','L2','','L4','','L6','','L8','',
          'T0','','T2','','T4','','T6','','T8','']
ylabels = ['','11','','13','','15','','17','']

#Same as 5, plots the left side of subplot
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(131)
ax.errorbar(spectype[notdelta], LV[notdelta], ls='', c='grey', alpha=0.2, fmt="v")
ax.errorbar(spectype[delta], LV[delta], yerr=ysig_b[delta], ls='', c='k', fmt="o")
for i in xrange(0, len(lmconf.chain), 100): #outputs the value for each chain iteration
    xs = np.arange(6,31)
    ys = lmconf.chain[i]['alpha'] + xs * lmconf.chain[i]['beta']
    ax.plot(xs, ys, color='#44B3C2', alpha=0.07)
alphaconf_av = lmconf.chain['alpha'].mean()
betaconf_av = lmconf.chain['beta'].mean()
ys = alphaconf_av + xs * betaconf_av
ax.plot(xs, ys, color='k')
ax.set_xlabel('Spectral Type')
ax.set_ylabel('log$_{10}$L$_V$ [erg s$^{-1}$ Hz$^{-1}$]')
ax.set_xlim(6,29)
ax.set_ylim(10,18)
ax.annotate('slope:' + str(betaconf_av) + '+/-' + str(lmconf.chain['beta'].std()), (7,17))
plt.xticks(np.arange(6,30,1), labels)
plt.yticks(np.arange(10,19,1), ylabels)

#Now the middle plot
ax = fig.add_subplot(132)
ax.scatter(spectype, LV, c='k', alpha=0.5)
# ax.errorbar(spectype, LV, xerr=xsig, yerr=ysig, ls=' ', alpha=0.5)
for i in xrange(0, len(lm.chain), 100): #outputs the value for each chain iteration
    xs = np.arange(6,31)
    ys = lm.chain[i]['alpha'] + xs * lm.chain[i]['beta']
    ax.plot(xs, ys, color='#44B3C2', alpha=0.07)
alpha_av = lm.chain['alpha'].mean()
beta_av = lm.chain['beta'].mean()
ys = alpha_av + xs * beta_av
ax.plot(xs, ys, color='k')
ax.set_xlabel('Spectral Type')
ax.set_ylabel('log$_{10}$L$_V$ [erg s$^{-1}$ Hz$^{-1}$]')
ax.set_xlim(6,29)
ax.set_ylim(10,18)
ax.annotate('slope:' + str(beta_av) + '+/-' + str(lm.chain['beta'].std()), (7,17))
plt.xticks(np.arange(6,30,1), labels)
plt.yticks(np.arange(10,19,1), ylabels)

#Now the right side plot, this has upper limits
ax = fig.add_subplot(133)
ax.errorbar(spectype[notdelta], LV[notdelta], yerr=ysig[notdelta],uplims=np.ones(sum(notdelta), dtype=bool), ls='', c='r', alpha=0.4, fmt="v")
ax.errorbar(spectype[delta], LV[delta], yerr=ysig_b[delta], ls='', c='k', fmt="o")
for i in xrange(0, len(lmcens.chain), 100): #outputs the value for each chain iteration
    xs = np.arange(6, 31)
    ys = lmcens.chain[i]['alpha'] + xs * lmcens.chain[i]['beta']
    ax.plot(xs, ys, color='#44B3C2', alpha=0.07)
alphacens_av = lmcens.chain['alpha'].mean()
betacens_av = lmcens.chain['beta'].mean()
ys = alphacens_av + xs * betacens_av
ax.plot(xs, ys, color='k')
ax.set_xlabel('Spectral Type')
ax.set_ylabel('log$_{10}$L$_V$ [erg s$^{-1}$ Hz$^{-1}$]')
ax.set_xlim(6,29)
ax.set_ylim(10,18)
ax.annotate('slope:' + str(betacens_av) + '+/-' + str(lmcens.chain['beta'].std()), (7,17))
plt.xticks(np.arange(6,30,1), labels)
plt.yticks(np.arange(10,19,1), ylabels)
fig.tight_layout()
fig.savefig('radio_linmixanalysis_upperlims.png')
plt.close(fig)
plt.show()

print alphacens_av, lmcens.chain['alpha'].std()
print betacens_av,  lmcens.chain['beta'].std()

print alphaconf_av, lmconf.chain['alpha'].std()
print betaconf_av,  lmconf.chain['beta'].std()

In [ ]:
# LV_Confirmed=LV[np.all([delta < 1], axis=0)]
# print LV_Confirmed

# LV_UL=LV[np.all([delta equal 1], axis =0)]
# print LV_UL

In [ ]:
import scipy
import scipy.stats
x = spectype
y = LV
h = plt.scatter(x, y, color='k')

dist_names = ['gamma', 'beta', 'rayleigh', 'norm', 'pareto']

for dist_name in dist_names:
    dist = getattr(scipy.stats, dist_name)
    param = dist.fit(y)
    pdf_fitted = dist.pdf(x, *param[:-2], loc=param[-2], scale=param[0]) * size
    plt.plot(pdf_fitted, label=dist_name)
    plt.xlim(6,29)
    plt.ylim(10,18)
plt.legend(loc='upper right')
plt.show()

In [ ]:
dist.pdf?

In [ ]:
#############################################################################################################
#Now trying with temperature on the x-axis

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import math
import linmix

#1. This section imports the necessary columns from the csv file into arrays
csv = np.genfromtxt ('RadioDetectionTable_Limits.csv', delimiter=",")
t_eff = csv[1:,13]
xsig = csv[1:,14]
LV = csv[1:,7]
ysig = csv[1:,8]
delta = csv[1:,10]


#Creating arrays that will specify only the sample values that have an effective temperature

len_t_eff = t_eff == 0
len_t_eff = np.logical_not(len_t_eff)

t_eff = t_eff[len_t_eff]
xsig = xsig[len_t_eff]
LV = LV[len_t_eff]
ysig = ysig[len_t_eff]
delta = delta[len_t_eff]
delta = delta.astype(int)

delta = delta == 0

In [ ]:
#2. Converts radio flux into log_10 space
LV=np.log10(LV)

In [ ]:
#3. This runs the mcmc code with delta indicating upper limits. 0=confirmed detections 1=null detections
lmcens  = linmix.LinMix(t_eff, LV, xsig, ysig, delta=delta, K=2)
lmcens.run_mcmc(silent=True)

In [ ]:
#4. This runs the new plots with upper limits factored in

#Defines error arrays
ysig_b=np.multiply(LV, 0.1)
ysig_upper= LV + ysig_b
ysig_lower= LV - ysig_b 
ysig_b=ysig_upper - ysig_lower
ysig_b=np.log10(ysig_b)

#Defines not delta to graph correct arrays
notdelta = np.logical_not(delta)
# labels = ['M6','','M8','','L0','','L2','','L4','','L6','','L8','',
#           'T0','','T2','','T4','','T6','','T8','']
ylabels = ['','11','','13','','15','']

#Now plots the data
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ax.errorbar(t_eff[notdelta], LV[notdelta], xerr=xsig[notdelta], yerr=ysig[notdelta], uplims=np.ones(sum(notdelta), dtype=bool), ls='', c='r', alpha=0.4, fmt="v")
ax.errorbar(t_eff[delta], LV[delta], xerr=xsig[delta], yerr=ysig_b[delta], ls='', c='k', fmt="o")
for i in xrange(0, len(lmcens.chain), 100): #outputs the value for each chain iteration
    xs = np.arange(500, 3000)
    ys = lmcens.chain[i]['alpha'] + xs * lmcens.chain[i]['beta']
    ax.plot(xs, ys, color='#44B3C2', alpha=0.07)
alphacens_av = lmcens.chain['alpha'].mean()
betacens_av = lmcens.chain['beta'].mean()
ys = alphacens_av + xs * betacens_av
ax.plot(xs, ys, color='k')
ax.set_xlabel('Effective Temperature (K)')
ax.set_ylabel('log$_{10}$L$_V$ [erg s$^{-1}$ Hz$^{-1}$]')
ax.set_xlim(500,3000)
ax.set_ylim(10,16)
ax.annotate('slope:' + str(betacens_av) + '+/-' + str(lmcens.chain['beta'].std()), (2900,15.35))
# plt.xticks(np.arange(6,30,1), labels)
plt.yticks(np.arange(10,16,1), ylabels)
plt.gca().invert_xaxis()
fig.tight_layout()
fig.savefig('radio_linmixanalysis_teff.png')
plt.close(fig)
plt.show()

In [2]:
#############################################################################################################
#Now trying with vsini on the x-axis

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import math
import linmix

#1. This section imports the necessary columns from the csv file into arrays
csv = np.genfromtxt ('RadioDetectionTable_Limits.csv', delimiter=",")
vsini = csv[1:,3]
xsig = csv[1:,16]
LV = csv[1:,7]
ysig = csv[1:,8]
delta = csv[1:,10]


#Creating arrays that will specify only the sample values that have an effective temperature

len_vsini = vsini == 0
len_vsini = np.logical_not(len_vsini)

vsini = vsini[len_vsini]
xsig = xsig[len_vsini]
LV = LV[len_vsini]
ysig = ysig[len_vsini]
delta = delta[len_vsini]
delta = delta.astype(int)

delta = delta == 0

In [3]:
#2. Converts radio flux into log_10 space
LV=np.log10(LV)

In [4]:
#3. This runs the mcmc code with delta indicating upper limits. 0=confirmed detections 1=null detections
lmcens  = linmix.LinMix(vsini, LV, xsig, ysig, delta=delta, K=2)
lmcens.run_mcmc(silent=False)


Iteration:  100
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 1.43607757  1.29677189  1.09980481  0.99606919  1.00235784  1.28749253]

Iteration:  200
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 1.07788663  1.08674731  1.03336493  0.99517251  0.996046    1.11426085]

Iteration:  300
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 1.06536221  1.07320057  1.01660178  0.99755877  0.99829231  1.07447983]

Iteration:  400
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 1.0458515   1.03720199  1.02417371  0.99873552  0.99807374  1.01894366]

Iteration:  500
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, eta)):
[ 1.03244149  1.03103982  1.04101215  0.99928034  0.99807468  1.01697992]

Iteration:  600
Rhat values for alpha, beta, log(sigma^2), mean(xi), log(var(xi)), atanh(corr(xi, e

In [5]:
#4. This runs the new plots with upper limits factored in

#Defines error arrays
ysig_b=np.multiply(LV, 0.1)
ysig_upper= LV + ysig_b
ysig_lower= LV - ysig_b 
ysig_b=ysig_upper - ysig_lower
ysig_b=np.log10(ysig_b)

#Defines not delta to graph correct arrays
notdelta = np.logical_not(delta)
# labels = ['M6','','M8','','L0','','L2','','L4','','L6','','L8','',
#           'T0','','T2','','T4','','T6','','T8','']
ylabels = ['11','','13','','15','']

#Now plots the data
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ax.errorbar(vsini[notdelta], LV[notdelta], xerr=xsig[notdelta], yerr=ysig[notdelta],uplims=np.ones(sum(notdelta), dtype=bool), ls='', c='r', alpha=0.4, fmt="v")
ax.errorbar(vsini[delta], LV[delta], xerr=xsig[delta], yerr=ysig_b[delta], ls='', c='k', fmt="o")
for i in xrange(0, len(lmcens.chain), 100): #outputs the value for each chain iteration
    xs = np.arange(0, 80)
    ys = lmcens.chain[i]['alpha'] + xs * lmcens.chain[i]['beta']
    ax.plot(xs, ys, color='#44B3C2', alpha=0.07)
alphacens_av = lmcens.chain['alpha'].mean()
betacens_av = lmcens.chain['beta'].mean()
ys = alphacens_av + xs * betacens_av
ax.plot(xs, ys, color='k')
ax.set_xlabel('vsini (m s$^{-1}$])')
ax.set_ylabel('log$_{10}$L$_V$ [erg s$^{-1}$ Hz$^{-1}$]')
ax.set_xlim(0,80)
ax.set_ylim(11,16)
ax.annotate('slope:' + str(betacens_av) + '+/-' + str(lmcens.chain['beta'].std()), (75,15.35))
# plt.xticks(np.arange(6,30,1), labels)
plt.yticks(np.arange(11,16,1), ylabels)
plt.gca().invert_xaxis()
fig.tight_layout()
fig.savefig('radio_linmixanalysis_vsini.png')
plt.close(fig)
plt.show()